# Webcrawling process

In [16]:
import pandas as pd
import requests
import warnings
# warnings.filterwarnings("ignore")

Generate Links

In [2]:
url_DOM = [f'https://www.the-numbers.com/market/{2005 + i}/top-grossing-movies' for i in range(0,21)]
url_INTL = [f'https://www.the-numbers.com/box-office-records/international/all-movies/cumulative/released-in-{2005 + i}' for i in range(0,21)]
url_WW = [f'https://www.the-numbers.com/box-office-records/worldwide/all-movies/cumulative/released-in-{2005 + i}' for i in range(0,21)]

In [3]:
def expand_paginated_urls(base_urls, start=101, step=100, max_pages=3):
  expanded = []
  years = range(2005,2026)
  for year, base in zip(years, base_urls):
    expanded.append(((year, base)))
    # Probe 101, 201, 301, ...
    for offset in range(start, start + step * max_pages, step):
      paged = f"{base}/{offset}"
      expanded.append((year,paged))
  return expanded

url_DOM_2 = expand_paginated_urls(url_DOM, start=101, step=100, max_pages=3)
url_INTL_2 = expand_paginated_urls(url_INTL, start=101, step=100, max_pages=3)
url_WW_2 = expand_paginated_urls(url_WW, start=101, step=100, max_pages=3)

## Get data tables 2005-2025

In [67]:
from collections import defaultdict
import requests

def get_table_from(urls=list,index=int,mkt=str,keep=list,links=False,to_csv=False):
  header = { #FAKE BROWSER
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
  }
  base_url = 'https://www.the-numbers.com'
  rows_removed = -2 if mkt == 'DOM' else None
  year_to_df = defaultdict(list)  # yr -> list[dataframes]
  s = requests.Session()

  # urls is expected to be an iterable of (year, url) pairs
  for yr, link in urls:
    try:
      r = s.get(link, headers=header, timeout=15)
      df = pd.read_html(r.text)[index]
      df = df[:rows_removed] if rows_removed is not None else df
      df['trunc'] = df['Movie'].astype(str).str.slice(0, 20)

      if links:
        df_link = pd.read_html(r.text, extract_links='body')[index]
        df_link['link'] = df_link['Movie'].apply(lambda v: f'{base_url}{v[1]}' if v[1] is not None else None)
        df_link['Movie'] = df_link['Movie'].apply(lambda v: v[0])
        df = pd.merge(df, df_link[['Movie', 'link']], on='Movie', how='left')

      df = df[keep] if keep else df
      year_to_df[yr].append(df)
    except Exception:
      continue

  # Concatenate all pages per year
  d = {}
  for yr, frames in year_to_df.items():
    if frames:
      d[f"df_{mkt}_{yr}"] = pd.concat(frames, axis=0, ignore_index=True)

  if to_csv == True:
    for name, df in d.items(): 
      df.to_csv(f'{mkt}/{name}.csv', index=False)
  return d

### Domestic:

In [5]:
# df_DOM = get_table_from(url_DOM_2[:],0,mkt='DOM',
# keep=['Movie','trunc','Release Date','Distributor','Genre'])
# df_DOM

### International:

In [6]:
# df_INTL = get_table_from(url_INTL_2[:],1,mkt='INTL',
# keep=['Movie','trunc','Share Of Number One Market','Number One Market'])
# df_INTL

### Worldwide:

In [ ]:
# df_WW = get_table_from(url_WW_2[:],1,mkt='WW',links=True,to_csv=True)

## Merge tables to `df_WW`
Phần này vẫn làm ở Python do phải điền thêm data thiếu..

In [63]:
import glob, re
import pandas as pd

# Load all year CSVs from folders and build per-year dataframes
def load_year_frames(folder, prefix):
  files = glob.glob(f"{folder}/{prefix}_*.csv")
  out = {}
  for path in files:
    m = re.search(r"_(\d{4})\.csv$", path)
    if not m:
      continue
    yr = int(m.group(1))
    try:
      df = pd.read_csv(path)
    except Exception:
      continue
    if 'trunc' not in df.columns and 'Movie' in df.columns:
      df['trunc'] = df['Movie'].astype(str).str.slice(0, 20)
    out[yr] = df
  return out

ww_by_year   = load_year_frames('WW',   'df_WW')
intl_by_year = load_year_frames('INTL', 'df_INTL')
dom_by_year  = load_year_frames('DOM',  'df_DOM')

merged_frames = []
for yr, ww0 in ww_by_year.items():
  ww = ww0.copy()
  if yr in intl_by_year:
    ww = pd.merge(
      ww,
      intl_by_year[yr][['trunc','Share Of Number One Market','Number One Market']],
      on='trunc', how='left'
    )
  if yr in dom_by_year:
    ww = pd.merge(
      ww,
      dom_by_year[yr][['trunc','Release Date','Distributor','Genre']],
      on='trunc', how='left'
    )
  ww['Year'] = yr
  if 'trunc' in ww.columns:
    ww = ww.drop(columns=['trunc'])
  merged_frames.append(ww)

# Concatenate all years
if merged_frames:
  df_WW_all = pd.concat(merged_frames, ignore_index=True)
else:
  df_WW_all = pd.DataFrame()

move_col = df_WW_all.pop('Rank')
df_WW_all.insert(9,'Rank',move_col)

move_col = df_WW_all.pop('link')
df_WW_all.insert(1,'link',move_col)

print(f"Years loaded - WW: {len(ww_by_year)}, INTL: {len(intl_by_year)}, DOM: {len(dom_by_year)}")
print(f"Final rows: {len(df_WW_all)}")

Years loaded - WW: 21, INTL: 21, DOM: 21
Final rows: 4829


In [64]:
df_WW_all.to_csv('WW2/WW_all.csv')

In [65]:
df_WW_all.tail()

,Movie,link,Worldwide Box Office,Domestic Box Office,International Box Office,Domestic Share,Share Of Number One Market,Number One Market,Release Date,Rank,Distributor,Genre,Year
4824,Io Sono La Fine Del Mondo,https://www.the-numbers.com/movie/Io-Sono-La-F...,"$10,164,861",NaN,"$10,164,861",NaN,100.0%,Italy,NaN,137,NaN,NaN,2025
4825,Three Kingdoms: Starlit Heroes (三国的星空第一部)…,https://www.the-numbers.com/movie/Three-Kingdo...,"$10,138,632",NaN,"$10,138,632",NaN,NaN,NaN,NaN,138,NaN,NaN,2025
4826,Na derevnyu dedushke (На деревню дедушке)…,https://www.the-numbers.com/movie/Na-derevnyu-...,"$10,124,608",NaN,"$10,124,608",NaN,100.0%,Russia (CIS),NaN,139,NaN,NaN,2025
4827,Dracula: A Love Tale,https://www.the-numbers.com/movie/Dracula-A-Lo...,"$10,092,908",NaN,"$10,092,908",NaN,87.2%,Russia (CIS),NaN,140,NaN,NaN,2025
4828,Red Silk (Красный шелк),https://www.the-numbers.com/movie/Red-Silk-(20...,"$10,016,598",NaN,"$10,016,598",NaN,82.4%,Russia (CIS),NaN,141,NaN,NaN,2025


In [66]:
df_WW_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4829 entries, 0 to 4828
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Movie                       4829 non-null   object
 1   link                        4829 non-null   object
 2   Worldwide Box Office        4829 non-null   object
 3   Domestic Box Office         3496 non-null   object
 4   International Box Office    4786 non-null   object
 5   Domestic Share              3496 non-null   object
 6   Share Of Number One Market  2868 non-null   object
 7   Number One Market           2955 non-null   object
 8   Release Date                3003 non-null   object
 9   Rank                        4829 non-null   int64 
 10  Distributor                 2980 non-null   object
 11  Genre                       3004 non-null   object
 12  Year                        4829 non-null   int64 
dtypes: int64(2), object(11)
memory usage: 490.6+ KB


## Get Movie Details of each film
This is the hardest part, not only does it takes time but there is also a risk of being blocked by the site (Error 403 Forbidden)...

In [ ]:
# Scrape Movie Details section (not in table format)
from bs4 import BeautifulSoup
import re
from datetime import datetime

def scrape_movie_details(url):
  """
  Scrape the Movie Details section from a the-numbers.com movie page
  Returns a dictionary with the structured data
  """
  header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
  }
  
  s = requests.Session()
  r = s.get(url, headers=header, timeout=15)
  soup = BeautifulSoup(r.text, 'html.parser')
  
  # Find the Movie Details section
  movie_details = {'link': url}
  
  # Look for the table with Movie Details
  details_table = soup.find('table', {'class': 'movie-details'})
  if not details_table:
    # Alternative: look for the section by text content
    details_section = soup.find('h2', string='Movie Details')
    if details_section:
      details_table = details_section.find_next('table')
  
  if details_table:
    rows = details_table.find_all('tr')
    for row in rows:
      cells = row.find_all(['td', 'th'])
      if len(cells) >= 2:
          key = cells[0].get_text(strip=True).replace('\xa0', ' ')
          value = cells[1].get_text(strip=True).replace('\xa0', ' ')
          
          # Clean up the key (remove colons and extra spaces)
          key = key.replace(':', '').strip()
          
          # Skip unwanted fields completely
          if key in ['Video Release', 'MPAA Rating', 'Franchise', 'Comparisons']:
              continue
          
          # Handle Production Countries and Languages separately
          if key == 'Production Countries':
              # Check if Languages data is mixed in
              if 'Languages:' in value:
                  parts = value.split('Languages:')
                  movie_details['Production Countries'] = parts[0].strip()
                  if len(parts) > 1:
                      movie_details['Languages'] = parts[1].strip()
              else:
                  movie_details['Production Countries'] = value
          elif key == 'Languages':
              movie_details['Languages'] = value
          else:
              # Store all other fields
              movie_details[key] = value
  
  # Extract earliest release date from Domestic and International releases
  release_dates = []
  
  # Extract dates from Domestic Releases
  if 'Domestic Releases' in movie_details:
      domestic_text = movie_details['Domestic Releases']
      # Look for date patterns like "February 14th, 2025"
      domestic_dates = re.findall(r'([A-Za-z]+ \d{1,2}(?:st|nd|rd|th)?, \d{4})', domestic_text)
      release_dates.extend(domestic_dates)
  
  # Extract dates from International Releases
  if 'International Releases' in movie_details:
      intl_text = movie_details['International Releases']
      # Look for date patterns like "January 29th, 2025"
      intl_dates = re.findall(r'([A-Za-z]+ \d{1,2}(?:st|nd|rd|th)?, \d{4})', intl_text)
      release_dates.extend(intl_dates)
  
  # Find the earliest date
  if release_dates:
      try:
          # Convert dates to datetime objects for comparison
          parsed_dates = []
          for date_str in release_dates:
              try:
                  # Handle ordinal suffixes (st, nd, rd, th)
                  clean_date = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', date_str)
                  parsed_date = datetime.strptime(clean_date, '%B %d, %Y')
                  parsed_dates.append(parsed_date)
              except:
                  continue
          
          if parsed_dates:
              earliest_date = min(parsed_dates)
              movie_details['Release Date'] = earliest_date.strftime('%B %d, %Y')
      except:
          pass
  
  # Remove the original release fields since we now have Release Date
  movie_details.pop('Domestic Releases', None)
  movie_details.pop('International Releases', None)
  
  return movie_details

In [133]:
# Convert scraped details to DataFrame
def details_to_series(details_dict):
  """
  Convert the scraped movie details dictionary to a DataFrame
  """
  if not details_dict:
    return pd.DataFrame()
  else:
    return pd.DataFrame([details_dict])

In [ ]:
# Collect all movie details as Series 
all_movie_details = []
urls = df_WW_all['link'][:2]
for url in urls:
  details = scrape_movie_details(url)
  if details:
    all_movie_details.append(details_to_series(details))
  print("Movie Details extracted:")
  for key, value in details.items():
    print(f"{key}: {value}")

# Concatenate all series to a dataframe
if all_movie_details:
  movie_details_df = pd.concat(all_movie_details, axis=0, ignore_index=True)
  move_col = movie_details_df.pop('Release Date')
  movie_details_df.insert(1,'Release Date', move_col)
  print("DataFrame shape:", movie_details_df.shape)
  print("\nDataFrame columns:", movie_details_df.columns.tolist())
  print("\n==========WEBSCRAPING COMPLETED==========\nDataFrame content:")
else:
  movie_details_df = pd.DataFrame()
movie_details_df.to_csv('WW2/movie_details.csv')
movie_details_df

Movie Details extracted:
link: https://www.the-numbers.com/movie/Star-Wars-Ep-III-Revenge-of-the-Sith#tab=summary
Running Time: 139 minutes
Keywords: Visual Effects,Good vs. Evil,Cyborg,Cloning,War,Betrayal,Death of a Spouse or Fiancée / Fiancé,Space Opera,Filmed in Shepperton Studios, Surrey, England,Filmed in Surrey, England,Filmed in England,Filmed in United Kingdom,Filmed in Elstree Studios, Hertfordshire, England,Filmed in Hertfordshire, England,Filmed in Fox Studios Australia, Sydney, Australia,Filmed in Sydney, Australia,Filmed in New South Wales, Australia,Filmed in Australia,Filmed in Phuket, Thailand,Filmed in Thailand,Filmed in Mount Etna, Italy,Filmed in Italy,Action Adventure
Source: Original Screenplay
Genre: Adventure
Production Method: Animation/Live Action
Creative Type: Science Fiction
Production/Financing Companies: Lucasfilm
Production Countries: United States
Languages: English
Release Date: May 18, 2005
Movie Details extracted:
link: https://www.the-numbers.com/mo

,Release Date,link,Running Time,Keywords,Source,Genre,Production Method,Creative Type,Production/Financing Companies,Production Countries,Languages
0,"May 18, 2005",https://www.the-numbers.com/movie/Star-Wars-Ep...,139 minutes,"Visual Effects,Good vs. Evil,Cyborg,Cloning,Wa...",Original Screenplay,Adventure,Animation/Live Action,Science Fiction,Lucasfilm,United States,English
0,"November 18, 2005",https://www.the-numbers.com/movie/Harry-Potter...,150 minutes,"Boarding School,Visual Effects,IMAX: DMR,Famil...",Based on Fiction Book/Short Story,Adventure,Animation/Live Action,Fantasy,"Warner Bros.,Heyday Films","United Kingdom,United States",English


In [132]:
# # Convert the scraped details to DataFrame
# # Test with Ne Zha 2
# url = 'https://www.the-numbers.com/movie/Ne-Zha-2-(2025-China)#tab=summary'
# details = scrape_movie_details(url)
# print("Movie Details extracted:")
# for key, value in details.items():
#   print(f"{key}: {value}")

# movie_details_df = details_to_dataframe(details)
# move_col = movie_details_df.pop('Release Date')
# movie_details_df.insert(0,'Release Date', move_col)
# print("DataFrame shape:", movie_details_df.shape)
# print("\nDataFrame columns:", movie_details_df.columns.tolist())
# print("\nDataFrame content:")
# movie_details_df